In [37]:
%pip install pmdarima

     ------------------------------------ 572.0/572.0 kB 642.0 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [126]:
import mediapipe as mp
import cv2
import numpy as np
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
from datetime import datetime
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split 
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from statsmodels.tsa.seasonal import seasonal_decompose
from datetime import timedelta

In [10]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [399]:
data = {"dates" : [],
        "values" : [],
        "spike/deep": []
        }

cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    counter = 0
    iter = 0
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass


        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )


        # Get coordinates
        R_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
        R_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
        R_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
        #Visibility Extraction
        R_V = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility + landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility + landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].visibility
        

        L_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
        L_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
        L_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
        #Visibility Extraction
        L_V = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility + landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility + landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility
               
        if R_V > L_V:
            angle = calculate_angle(R_hip, R_knee, R_ankle)
        else:
            angle = calculate_angle(L_hip, L_knee, L_ankle)
        counter+=1

                    # Visualize angle
        cv2.putText(image, str(angle), 
                        #tuple(np.multiply(L_knee, [1280, 720]).astype(int)), 
                        tuple(np.multiply(L_knee, [640, 480]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA

                            )
        if counter == 4:
            counter = 0
            iter += 1

        """ 
        c = 0
        avg_y = 0
        for i in range(11):
            if landmarks[i+11].visibility > 0.9:
                avg_y += landmarks[i+11].y
                c += 1
        avg_y / c
        """

        avg_y = (landmarks[11].y + landmarks[13].y + landmarks[15].y)/3

        if angle > 160:
            data["spike/deep"].append(-1)
            data["values"].append(avg_y)
            data["dates"].append(datetime.today().strftime("%H:%M:%S:%f"))
        elif angle < 110:
            data["spike/deep"].append(1)
            data["values"].append(avg_y)
            data["dates"].append(datetime.today().strftime("%H:%M:%S:%f"))
        else:
            data["spike/deep"].append(0)
            data["values"].append(avg_y)
            data["dates"].append(datetime.today().strftime("%H:%M:%S:%f"))
            
                
        
        cv2.imshow('Mediapipe Feed', image)
        
        if (cv2.waitKey(10) & 0xFF == ord('q') | (iter == 100)):
            break

    cap.release()
    cv2.destroyAllWindows()

In [400]:
sum = 0
for i in range(len(data)-1):
    d1 = datetime.strptime(data["dates"][i], "%H:%M:%S:%f")
    d2 = datetime.strptime(data["dates"][i+1], "%H:%M:%S:%f")
    delta = d2 - d1
    delta_ms = round(delta.microseconds,0) / 1000
    sum = sum + delta_ms
avg_ms = sum // len(data)-1
print(avg_ms)

52.0


In [401]:
for i in range(70):
    d1 = datetime.strptime(data["dates"][-1], "%H:%M:%S:%f")
    sum = d1 + timedelta(milliseconds=avg_ms)
    data["dates"].append(datetime.strftime(sum,"%H:%M:%S:%f"))
    data["values"].append(0)
    data["spike/deep"].append(0)
    print(sum)

1900-01-01 16:26:57.136752
1900-01-01 16:26:57.188752
1900-01-01 16:26:57.240752
1900-01-01 16:26:57.292752
1900-01-01 16:26:57.344752
1900-01-01 16:26:57.396752
1900-01-01 16:26:57.448752
1900-01-01 16:26:57.500752
1900-01-01 16:26:57.552752
1900-01-01 16:26:57.604752
1900-01-01 16:26:57.656752
1900-01-01 16:26:57.708752
1900-01-01 16:26:57.760752
1900-01-01 16:26:57.812752
1900-01-01 16:26:57.864752
1900-01-01 16:26:57.916752
1900-01-01 16:26:57.968752
1900-01-01 16:26:58.020752
1900-01-01 16:26:58.072752
1900-01-01 16:26:58.124752
1900-01-01 16:26:58.176752
1900-01-01 16:26:58.228752
1900-01-01 16:26:58.280752
1900-01-01 16:26:58.332752
1900-01-01 16:26:58.384752
1900-01-01 16:26:58.436752
1900-01-01 16:26:58.488752
1900-01-01 16:26:58.540752
1900-01-01 16:26:58.592752
1900-01-01 16:26:58.644752
1900-01-01 16:26:58.696752
1900-01-01 16:26:58.748752
1900-01-01 16:26:58.800752
1900-01-01 16:26:58.852752
1900-01-01 16:26:58.904752
1900-01-01 16:26:58.956752
1900-01-01 16:26:59.008752
1

In [402]:
#To Dataframe
df = pd.DataFrame(data = data)

#df_raw = df_raw.drop(index=range(28))

#without Outliers
#df = df_raw[(df_raw['values'] > df_raw['values'].quantile(q=0.75)) |
#            (df_raw['values'] < df_raw['values'].quantile(q=0.25)) ]

In [403]:
df.to_csv("Squats_Unfiltered.csv" , index = False)

In [404]:
%matplotlib qt
df.plot(kind='line', y = "values", x = "dates", style='-o')

<AxesSubplot:xlabel='dates'>

In [406]:
res = seasonal_decompose(df["values"][:len(df)-100],model = "additive",period=7)
res.plot();

In [407]:
############## NO SPIKES / DEEPS ##################
auto_arima(df["values"][:len(df)-100], trace=True, seasonal=True).summary()

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=-2238.469, Time=0.68 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=-704.987, Time=0.09 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=-2138.035, Time=0.10 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=-1211.298, Time=0.13 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=343.722, Time=0.03 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=-2268.007, Time=0.45 sec
 ARIMA(0,0,2)(0,0,0)[0] intercept   : AIC=-1579.339, Time=0.42 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=-2227.001, Time=0.33 sec
 ARIMA(1,0,3)(0,0,0)[0] intercept   : AIC=-2293.145, Time=0.70 sec
 ARIMA(0,0,3)(0,0,0)[0] intercept   : AIC=-1799.575, Time=0.37 sec
 ARIMA(2,0,3)(0,0,0)[0] intercept   : AIC=-2271.928, Time=0.79 sec
 ARIMA(1,0,4)(0,0,0)[0] intercept   : AIC=-2334.528, Time=0.96 sec
 ARIMA(0,0,4)(0,0,0)[0] intercept   : AIC=-1952.427, Time=0.58 sec
 ARIMA(2,0,4)(0,0,0)[0] intercept   : AIC=-2293.559, Time=1.09 sec
 ARIMA(1,0,5)(0,0,0)[0

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  423
Model:               SARIMAX(2, 0, 5)   Log Likelihood                1243.323
Date:                Thu, 09 Mar 2023   AIC                          -2468.646
Time:                        16:28:38   BIC                          -2432.219
Sample:                             0   HQIC                         -2454.253
                                - 423                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0101      0.001      7.430      0.000       0.007       0.013
ar.L1          1.9260      0.022     86.291      0.000       1.882       1.970
ar.L2         -0.9549      0.022    -43.657      0.000      -0.998      -0.912
ma.L1         -0.9588      0.065    -14.807      0.000      -1.086      -0.832
ma.L2          0.1813      0.095      1.907      0.056      -0.005       0.368
ma.L3          0.1739      0.083      2.103      0.035       0.012       0.336
ma.L4          0.0820      0.071      1.159      0.246      -0.057       0.221
ma.L5         -0.0408      0.045     -0.901      0.368      -0.130       0.048
sigma2         0.0002   9.42e-06     17.087      0.000       0.000       0.000
===================================================================================
Ljung-Box (L1) (Q):                   2.13   Jarque-Bera (JB):               199.29
Prob(Q):                              0.14   Prob(JB):                         0.00
Heteroskedasticity (H):               0.68   Skew:                             0.72
Prob(H) (two-sided):                  0.02   Kurtosis:                         6.04
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [ ]:
for i in range(100):
    index = 101 - i
    data["values"][len(df)-index:] = data["values"][len(df)-index-17:]


In [ ]:
df

In [390]:
############## NO SPIKES / DEEPS ##################
train_df = df.iloc[:len(df)-100]
test_df = df.iloc[len(df)-100:]
model = SARIMAX(train_df["values"], order=(35,0, 0), trend='c')
res = model.fit()
start = len(train_df)
end = len(train_df) + len(test_df) -1
prediction = res.predict(start, end).rename("prediction")
ax = df["values"].plot(legend=True,figsize=(16,8))
prediction.plot(legend=True)


c:\Users\mosac\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<AxesSubplot:>

In [353]:
############## NO SPIKES / DEEPS ##################
train_df = df.iloc[:len(df)-100]
test_df = df.iloc[len(df)-100:]
model = SARIMAX(train_df["spike/deep"], order=(20,0,1))
res = model.fit()
start = len(train_df)
end = len(train_df) + len(test_df) -1
prediction = res.predict(start, end).rename("prediction")
ax = df["spike/deep"].plot(legend=True,figsize=(16,8))
prediction.plot(legend=True)


<AxesSubplot:>

In [329]:
############## NO SPIKES / DEEPS ##################
from statsmodels.tools.eval_measures import rmse
accuracy_no_deeps = 1- rmse(test_df["values"], prediction)
print(accuracy_no_deeps)

0.6427576846799165


In [269]:
############## WITH SPIKES / DEEPS ##################
auto_arima(df["values"][:len(df)-100], exogenous=df["spike/deep"], seasonal=True , start_q=0,max_q= 1, trace  =True).summary()

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=-1769.123, Time=0.59 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=-567.943, Time=0.07 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=-1695.878, Time=0.05 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=-1012.066, Time=0.27 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=331.628, Time=0.03 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=-1781.975, Time=0.34 sec
 ARIMA(0,0,2)(0,0,0)[0] intercept   : AIC=-1295.928, Time=0.20 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=-1770.561, Time=0.28 sec
 ARIMA(1,0,3)(0,0,0)[0] intercept   : AIC=-1823.179, Time=0.38 sec
 ARIMA(0,0,3)(0,0,0)[0] intercept   : AIC=inf, Time=0.45 sec
 ARIMA(2,0,3)(0,0,0)[0] intercept   : AIC=inf, Time=1.01 sec
 ARIMA(1,0,4)(0,0,0)[0] intercept   : AIC=-1857.999, Time=0.52 sec
 ARIMA(0,0,4)(0,0,0)[0] intercept   : AIC=-1575.606, Time=0.50 sec
 ARIMA(2,0,4)(0,0,0)[0] intercept   : AIC=inf, Time=1.16 sec
 ARIMA(1,0,5)(0,0,0)[0] intercept   : AI

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  375
Model:               SARIMAX(1, 0, 5)   Log Likelihood                 942.525
Date:                Thu, 09 Mar 2023   AIC                          -1869.049
Time:                        14:50:20   BIC                          -1837.634
Sample:                             0   HQIC                         -1856.577
                                - 375                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0149      0.013      1.175      0.240      -0.010       0.040
ar.L1          0.9559      0.032     30.154      0.000       0.894       1.018
ma.L1          0.2676      0.108      2.488      0.013       0.057       0.478
ma.L2          0.2343      0.067      3.495      0.000       0.103       0.366
ma.L3          0.3576      0.048      7.522      0.000       0.264       0.451
ma.L4          0.3231      0.062      5.223      0.000       0.202       0.444
ma.L5          0.1514      0.085      1.786      0.074      -0.015       0.318
sigma2         0.0004   1.24e-05     30.066      0.000       0.000       0.000
===================================================================================
Ljung-Box (L1) (Q):                   0.46   Jarque-Bera (JB):              9920.20
Prob(Q):                              0.50   Prob(JB):                         0.00
Heteroskedasticity (H):               0.80   Skew:                             2.74
Prob(H) (two-sided):                  0.21   Kurtosis:                        27.60
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [377]:
############## WITH SPIKES / DEEPS ##################
train_df = df.iloc[:len(df)-100]
test_df = df.iloc[len(df)-100:] 
model = SARIMAX(endog=train_df["values"], order=(30,1, 0), exog=train_df["spike/deep"])
res = model.fit()
start = len(train_df)
end = len(train_df) + len(test_df) -1
prediction = res.predict(start, end,exog = test_df["spike/deep"]).rename("prediction")
ax = df["values"].plot(legend=True,figsize=(16,8))
prediction.plot(legend=True)

c:\Users\mosac\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<AxesSubplot:>

In [226]:
############## WITH SPIKES / DEEPS ##################

# Need to make sure to truncate it in the right place
prediction

375    0.224689
376    0.223349
377    0.220437
378    0.219620
379    0.218325
380    0.217619
381    0.216676
382    0.216147
383    0.215631
384    0.215050
385    0.214590
386    0.214226
387    0.213938
388    0.213709
389    0.213529
390    0.213386
391    0.213272
392    0.213182
393    0.213111
394    0.213055
395    0.213010
396    0.212975
397    0.212947
398    0.212925
399    0.212908
400    0.212894
401    0.212883
402    0.212874
403    0.212867
404    0.212862
Name: prediction, dtype: float64

In [99]:
accuracy = 1- rmse(test_df["values"], prediction)
print(accuracy)

0.9173200783498131
